<h2><center>Residential Energy Consumption Prediction using Regression and Ensemble Methods!</center></h2>

### Table of Contents

1. [Introduction](#1)

2. [Project Objectives](#2)

3. [Loading Libraries and Reading Data](#3)<br>
    a. [Loading Libraries](#31)<br>
    b. [Version of Installed Python Library](#32)<br>
    c. [Loading Data](#33)<br>

4. [Exploratory Data Analysis (EDA)](#4)<br>
    a. [Data Dimensionality](#41)<br>
    b. [Data Types](#42)<br>
    c. [Summary Statistics](#43)<br>
    d. [Check for Missing Values](#44)<br>
    e. [Explore Target Variable and Extract Important Features](#45)<br>
    f. [Detect Outliers and Anomalies](#46)<br>
    g. [Explore Additional Predictor Variables](#47)<br>
    
5. [Data Transformation and Preprocessing](#5)<br>
    a. [Data Transformation](#51)<br>
    &emsp;i. &nbsp;[Combining/Merging Predictor Features](#511)<br>
    &emsp;ii. [Combining/Merging Levels with Low Frequency in Discrete Predictor Features](#512)<br>
    
    b. [Data Preprocessing](#52)<br>
    &emsp;i.&ensp; [Removing Predictor Features with high 'Not Applicable' Values](#521)<br>
    &emsp;ii.&nbsp; [Removing Imputation Flags](#522)<br>
    &emsp;iii. [Removing Duplicate Features](#523)<br>
    &emsp;iv.&nbsp;[Removing Outliers](#524)<br>

6. [Feature Engineering](#6)<br>
    a. [Exploratory Feature Reduction](#61)<br>
    b. [Feature Selection](#62)<br>
    &emsp;i. &nbsp;[Find Features with Single Unique Value](#621)<br>
    &emsp;ii. [Find Collinear Features](#622)<br>
    &emsp;iii. [Find Features with Zero Importance using GBM](#623)<br>
    &emsp;iv.&nbsp;[Find Features with Low Importance](#624)<br>
    &emsp;v.&nbsp;[Removing Features](#625)<br>

7. [Model Development & Comparison](#7)<br>
    a. [Building Baseline Models with Default Params](#71)<br>
    b. [Hyperparameter Tuning & Model Comparison](#72)<br>
    c. [Model Evaluation on Unseen Data](#73)

<a id="1"></a>
### 1. Introduction

Every four years, [EIA](https://www.eia.gov/consumption/residential/about.php) administers the Residential Energy Consumption Survey (RECS) to a nationally representative sample of housing units across the United States to collect energy characteristics data on the housing unit, usage patterns, and household demographics. $^{1}$

This project focuses on 2009 RECS survey data which represents the 13th iteration of the RECS program. First conducted in 1978, the Residential Energy Consumption Survey is a national sample survey that collects energy-related data for housing units occupied as a primary residence and the households that live in them. 2009 data were collected from 12,083 households selected at random using a complex multistage, area-probability sample design. The sample represents 113.6 million U.S. households, the Census Bureau’s statistical estimate for all occupied housing units in 2009 derived from their American Community Survey (ACS). $^{1}$

<a id="2"></a>
### 2. Project objectives

This project utilizes applied machine learning methods, Regression as well as Ensemble methods, on the 2009 RECS dataset to achieve the following three objectives:

- __Description:__ Describing/Exploring the set of features with the strong statistical associations with target variable Annual Electricity Usage (in kWh)

- __Feature Selection:__ Showcasing different feature selection methods to select the most critical features for electricity consumption;

- __Prediction:__ – Applying different machine learning models to measure and compare predictive performance of the selected features


The target variable is "KWH" which stands for kilowatt-hour.

<a id="3"></a>
### 3. Loading Libraries and Reading Data
<a id="31"></a>
#### a. Loading Libraries
Let's start by importing the libraries we need

In [1]:
#!pip install --upgrade "pandas==2.2.2" "numpy==2.2.0" "scipy==1.13.0" --force-reinstall --quiet
#import os
#os.kill(os.getpid(), 9)  # Restart runtime


In [2]:
import pandas as pd

# 🩹 Restore deprecated methods for compatibility
if not hasattr(pd.DataFrame, 'append'):
    def _append(self, other, ignore_index=False, verify_integrity=False, sort=False):
        from pandas import concat, DataFrame
        if isinstance(other, dict): other = DataFrame([other])
        elif not isinstance(other, DataFrame): other = DataFrame(other)
        return concat([self, other], ignore_index=ignore_index, verify_integrity=verify_integrity, sort=sort)
    pd.DataFrame.append = _append

if not hasattr(pd.Index, '_format_flat'):
    pd.Index._format_flat = lambda self, include_name=False: self.tolist()



In [3]:
import pandas as pd
from pandas import set_option
from pandas import get_dummies
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import scipy.stats

from sklearn import model_selection, preprocessing, metrics
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import scale, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA
import sklearn
import warnings
warnings.filterwarnings("ignore")

from feature_selector import FeatureSelector

#XGBoost libraries
import xgboost as xgb

ModuleNotFoundError: No module named 'feature_selector'

<a id="32"></a>
#### b. Version of Installed Python Library

For the purpose of reproducibility of this analysis, displaying the versions of installed libraries

In [ ]:
import pandas as pd

def _append(self, other, ignore_index=False, verify_integrity=False, sort=False):
    from pandas import concat, DataFrame
    if isinstance(other, dict): other = DataFrame([other])
    elif not isinstance(other, DataFrame): other = DataFrame(other)
    return concat([self, other], ignore_index=ignore_index, verify_integrity=verify_integrity, sort=sort)

pd.DataFrame.append = _append


In [ ]:
print('matplotlib: {}'.format(matplotlib.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('xgboost: {}'.format(xgb.__version__))


<a id="33"></a>
#### c. Loading Data

In [ ]:
DATA_PATH = 'data'
FILE_NAME = 'recs2009_public.csv'
df = pd.read_csv(os.path.join(DATA_PATH, FILE_NAME), encoding = 'utf-8-sig')

<a id="4"></a>
### 4. Exploratory Data Analysis (EDA)

In the EDA phase, we will perform initial investigations on the dataset to check the following:
- data dimensionality (or shape) alongwith observing first few observations of the dataset
- data types (whether categorical or numerical)
- generate summary statistics
- missing values
- explore our target variable _'KWH'_ and its possible predictors from the dataset
- detect outliers and anomalies
- explore additional predictor variables

<a id="41"></a>
#### a. Data Dimensionality

In [ ]:
import pandas as pd

# Patch for broken IPython display in Colab
if not hasattr(pd.Index, '_format_flat'):
    pd.Index._format_flat = lambda self, include_name=False: self.tolist()


Let's start by glancing first few observations of the RECS dataset alongwith its dimensionality

In [ ]:
# Set pandas to display all columns when printing DataFrames
# df.shape[1] gives the number of columns in the DataFrame
pd.options.display.max_columns = df.shape[1]


# Display the first 5 rows of the DataFrame
# Useful for getting a quick look at the data: column names, sample values, data types, etc.
df.head()

In [ ]:
print(f"Shape of the dataset df_recs is {df.shape}") #rows, columns

As can be seen above, RECS 2009 dataset consists of 12,083 observations and 940 features. Let's explore how many of these features are categorical vs numerical

<a id="42"></a>
#### b. Data Types

In [ ]:
num_features = df.dtypes[df.dtypes != "object"].index
print("Number of Numerical features: ", len(num_features))

cat_features = df.dtypes[df.dtypes == "object"].index
print("Number of Categorical features: ", len(cat_features))

`df.info()` method prints a concise summary of a dataframe including count of index data type, number of columns. `df.describe()` method prints out the descriptive statistics including mean, median (i.e. Second Quartile, Q2 depicted by 50% in the summary table below), standard deviation, range, Q1 and Q3 quartile values. As can be seen below, RECS 2009 dataset consists of three different column data types:
- _float64_ data type with 50 columns
- _int64_ data type with 885 columns, and
- _object_ data type with 5 columns

The count of column data types is consistent with the count of numerical and categorical features we calculated above. Count of Numerical features we calculated above is actually sum of number of _float64_ and _int64_ data type columns i.e. (50 + 885 = 935).

The number of records, 12083 is consistent with what we found using `df.shape`. We don't have any NA values in the dataset.

In [ ]:
 df.info()

<a id="43"></a>
#### c. Summary Statistics

In [ ]:
df.describe()

**Key Observations:**
- Mean value is different (i.e. more or less) than median value for columns HDD65, CDD65, HDD30YR, CDD30YR, TOTSQFT, KWH, CDD80, OA_LAT
- Large difference in 75th percentile and maximum value for columns HDD65, CDD65, HDD30YR, CDD30YR, TOTSQFT, KWH, CDD80, OA_LAT
- Thus, these observations mean that there are outier values in our dataset

Now, let's check the summary statistics for categorical features

In [ ]:
df.describe(include=['object'])

In the table above, we can see the number of unique values as well as the top value and it's frequency for all the categorical features. Upon closely looking at features 'NOCRCASH' and 'NKRGALNC', we can observe that the topmost value is -2 with a frequency of 9,958 in each of these features. This doesn't seem right as -2 is of _int64_ data type and we previously found these features to be categorical in nature. Hence we will now check the counts of unique values for each of these features to see if there was any data entry error

In [ ]:
df['NOCRCASH'].value_counts()

In [ ]:
df['NKRGALNC'].value_counts()

We can observe from the above value counts for features 'NOCRCASH' and 'NKRGALNC' that there is '.' (period symbol) in two observations for each of these features. We assume that these two observations with '.' value might be a data error while recording and saving the data.

In addition, it can be noticed that the value '-2' appears twice with a frequency count of 9958 and 2028 in features 'NOCRCASH' and 'NKRGALNC'. There might be a trailing or leading space after -2 value and hence we see -2 twice for each of the variables. The actually count of -2 i.e. Not Applicable value in each of these features is 11986 (9958 + 2028), which is more than 99% of the observations in the RECS 2009 dataset. Since more than 99% of the observations in 'NOCRCASH' and 'NKRGALNC' are actually 'Not Applicable' values, hence we can conclude that there is not enough variance in values of features 'NOCRCASH' and 'NKRGALNC' and thus these features would have no predicitve power over the outcome variable and we can safely drop these features from the dataset in the data preprocessing section.

Let's check the unique value counts for the remaining categorical features

In [ ]:
cat_features_remaining = ['METROMICRO', 'UR', 'IECC_Climate_Pub']

In [ ]:
for col in cat_features_remaining:
    print(df[col].value_counts(), '\n')

We can also check how values in each of the categorical features are distributed using argument `normalize = True` inside method `value_counts()` distribution of the unique value counts

In [ ]:
for col in cat_features_remaining:
    print(round(df[col].value_counts(normalize= True)*100), '\n')

85% of the responses (n = 10302) were recorded from housing units in census metropolitan area where as 9% of the responses (n = 1109) were recorded from housing units in census micropolitan area. On the contrary, 80% of the surveyed housing units (n = 9656) were from urban area whereas rest 20% (n = 2427) were from rural area

<a id="44"></a>
#### d. Check for Missing Values

Next, we check dataset for any missing values in it

In [ ]:
df.isnull().sum().sum()

Upon checking dataset for null values using method `df.isnull()`, we did not find any missing values in the dataset. However, looking into the RECS 2009 survey codebook, it was observed that most of the features had a response code of -2 i.e. 'Not Applicable' which meant that the feature being measured doesn't apply to survey respondent's housing, consumption and expenditure characterstics. Hence, we will now check how many features in total were marked -2 i.e. 'Not Applicable' by majority of the survey respondents

Let's check how many features in total were marked -2 i.e. 'Not Applicable' in more than 95% of the observations

In [ ]:
df_exclude_object_dtype = df.select_dtypes(exclude=['object'])
na_col_names = df_exclude_object_dtype.columns[((df_exclude_object_dtype == -2).sum()) > round(df.shape[0]*0.95)]
print('{} features were marked as -2 i.e. "Not Applicable" by majority (95%) of the respondents'.format(len(na_col_names)),"\n" "Here's the name of the features:", na_col_names)

72 features were found to have NA i.e. 'Not Applicable' values in more than 95% of the observations. Since, these features have majority of the data values as 'NA', it can be safely concluded that these features would have negligible or almost no predictive power over the outcome variable. In the later part of this section, we will check if these features correlate with our target variable _'KWH'_ in order to make a decision as to whether these features are to be retained or dropped from final set of feature dataset

<a id="45"></a>
#### e. Explore Target Variable and Extract Important Features

Let's explore our target variable _'KWH'_ and extract important features which are highly correlated with the target variable

In [ ]:
df['KWH'].describe()

Few key insights by looking at summary statistics of the dependent variable are as follows:
- Target variable _'KWH'_ is numeric in nature
- The mean value of target variable is ~ 11288 KWH and a standard deviation of ~ 7641 KWH which means the distance between observations and the mean value of KWH is considerably higher
- 50% of the KWH values lies between 17 - 9623 KWH, whereas 75% the values lies between 17 - 14765 KWH
- The min and max value for target variable are 17 and 150254 KWH

In [ ]:
fig, ax = plt.subplots(figsize=[10,6])
ax.set_xlim(0,90000)
sns.distplot(df['KWH'],ax=ax, bins=100).set(title = 'KWH Distribution')

It can be seen from the graph above that the distribution of target variable is positively skewed. This means that the outliers of the distribution are further towards the right. Let's create box plot (i.e. box and whisker diagram) for our target variable to see the range of values in a more intutive way

In [ ]:
sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(x=df['KWH']).set(title = 'KWH Boxplot')

As previously observed from the distribution graph of target variable, we can clearly see in the box plot above that almost all the observations for energy consumption are under 80,000 KWH except for just one extreme outlier value above KWH 140,000. We will take care of the outlier values in the data preprocessing section

In the distribution plot above for our target variable, we observed that the distribution was right or positively skewed. Hence, to tackle issue of skewness, let's apply the log transformation on our target variable

In [ ]:
df['KWHlog'] = np.log(df['KWH'])
fig, ax = plt.subplots(figsize=[10,6])
sns.distplot(df['KWHlog'],ax=ax).set(title = 'KWHlog Distribution')

We can observe from the distribution graph above that log transformation has brought the distribution close to normal or bell shape curve. In other words, we can say that the log transformation reduced the skeness of our original data and hence the statistical analysis results from log transformed data will be more accurate or valid.

To extract important predictors of our target variable 'KWH', we will find correlation between target variable 'KWH' and all predictor variables and filter the predictor variables by keeping the threshold correlation value of abs(0.4), meaning the predictor variables with a correlation of abs(0.4) with target variable 'KWH' will be shown.

In [ ]:
# Select only numeric columns from the DataFrame
df_numeric = df.select_dtypes(include=['number'])

# Compute the correlation matrix
corr = df_numeric.corr()

# Compute absolute correlation with the target variable 'KWH'
corr_target = abs(corr['KWH'])

# Select features with correlation > 0.4
relevant_features = corr_target[corr_target > 0.4]

# Sort features by correlation value
relevant_features = relevant_features.sort_values(ascending=False)

# Print summary
print("{} features were found to have correlation value of 0.4 or more with our target variable 'KWH'".format(len(relevant_features)))
print('----------------------------------------------------------------------------------------')
print('These features are: \n{}'.format(list(relevant_features.index)))


Let's now check which are the top 20 predictors for tagret variable 'KWH' on the basis of the results of correlation analysis

In [ ]:
relevant_features.sort_values(ascending = False)[:21]

Let's plot the 'KWH' correlation matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Keep only numeric columns
df_numeric = df.select_dtypes(include=['number'])

# Step 2: Select top 33 features most correlated with 'KWH'
k = 33
cols = df_numeric.corr().nlargest(k, 'KWH')['KWH'].index

# Step 3: Compute correlation matrix for selected features
cm = df_numeric[cols].corr()

# Step 4: Plot heatmap
plt.figure(figsize=(30,18))
sns.heatmap(cm, annot=True, cmap='viridis')
plt.title(f"Top {k} Features Correlated with KWH", fontsize=20)
plt.show()


Following may be observed from the above graph:
- green and yellow color represents strong positive correlation whereas blue and purple color represents very weak positive or negative correlation
- KWH has moderate to strong positive correlation (> 0.4) with all top 32 important predictors
- Many predictor variables have high correlation with each other (such as BTUELCOL, KWHCOL, TOTALBTUCOL), thus indicating the presence of multicollinearity among predictor variables/features. We will explore collinearity among predictor features in detail in the feature selection section

Let's now explore how does these predictor variables affect the target variable 'KWH' by graphing a scatter plot between target variable and a few predictor variables

**How does 'BTUEL' i.e. Electricity Usage in BTU relates with the target variable 'KWH'?**

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x='BTUEL',y='KWH',data=df)
ax.set_ylim(ymin=0)
plt.xlabel('Electricity Usage in BTU')
plt.ylabel('Electricity Usage in KWH')

We can see from the scatterplot that KWH and BTUEL is perfectly correlated. Infact, it is a duplicate variable indicating total electricity usage in different units, thousand BTU in variable BTUEL and kilowatt-hours in variable 'KWH'. Hence, in the data prepracessing section we will remove these duplicates. There are few other features in the RECS dataset which are actually either duplicates or calculated by summing up the one or more predictor features. These are as follows:

- KWH = KWHSPH + KWHCOL + KWHWTH + KWHRFG + KWHOTH<br>
- BTUEL = BTUELSPH + BTUELCOL + BTUELWTH + BTUELRFG + BTUELOTH<br>
- DOLLAREL = DOLELSPH + DOLELCOL + DOLELWTH + DOLELRFG + DOLELOTH<br>
- CUFEETNG = CUFEETNGSPH + CUFEETNGWTH + CUFEETNGOTH<br>
- BTUNG = BTUNGSPH + BTUNGWTH + BTUNGOTH<br>
- DOLLARNG = DOLNGSPH + DOLNGWTH + DOLNGOTH<br>
- GALLONLP = GALLONLPSPH + GALLONLPWTH + GALLONLPOTH<br>
- BTULP = BTULPSPH + BTULPWTH + BTULPOTH<br>
- DOLLARLP = DOLLPSPH	+ DOLLPWTH + DOLLPOTH<br>
- GALLONFO = GALLONFOSPH + GALLONFOWTH + GALLONFOOTH<br>
- BTUFO = BTUFOSPH + BTUFOWTH + BTUFOOTH<br>
- DOLLARFO = DOLFOSPH + DOLFOWTH + DOLFOOTH<br>
- GALLONKER = GALLONKERSPH + GALLONKERWTH + GALLONKEROTH<br>
- BTUKER = BTUKERSPH + BTUKERWTH + BTUKEROTH<br>
- DOLLARKER = DOLKERSPH + DOLKERWTH + DOLKEROTH<br>
- TOTALBTU = TOTALBTUSPH + TOTALBTUCOL + TOTALBTUWTH + TOTALBTURFG + TOTALBTUOTH<br>
- TOTALDOL = TOTALDOLSPH + TOTALDOLCOL + TOTALDOLWTH + TOTALDOLRFG + TOTALDOLOTH<br>
- TOTALBTUSPH = BTUELSPH + BTUNGSPH + BTULPSPH + BTUFOSPH + BTUKERSPH<br>
- TOTALBTUCOL = BTUELCOL<br>
- TOTALBTUWTH = BTUELWTH + BTUNGWTH + BTULPWTH +  + BTUFOWTH + BTUKERWTH<br>
- TOTALBTURFG = BTUELRFG<br>
- TOTALBTUOTH = BTUELOTH + BTUNGOTH + BTULPOTH + BTUFOOTH + BTUKEROTH<br>
- TOTALDOLSPH = DOLELSPH + DOLNGSPH + DOLLPSPH + DOLFOSPH + DOLKERSPH<br>
- TOTALDOLCOL = DOLELCOL<br>
- TOTALDOLWTH = DOLELWTH + DOLNGWTH + DOLLPWTH + DOLFOWTH + DOLKERWTH<br>
- TOTALDOLRFG = DOLELRFG<br>
- TOTALDOLOTH = DOLELOTH + DOLNGOTH + DOLLPOTH + DOLFOOTH + DOLKEROTH<br>

We will talk about these in detail in data preprocessing section


**Now, let's check how does Total Cooled Square Footage (TOTCSQFT), Total heated square footage (TOTHSQFT) and Total Number of Rooms in Housing (TOTROOMS) relates with the target variable 'KWH'?**

In [ ]:
#f = plt.figure(figsize = (10,3))
fig, axs = plt.subplots(1, 3, figsize=(24, 8))
axs[0].scatter(x='TOTCSQFT',y='KWH',data=df)
axs[1].scatter(x='TOTHSQFT',y='KWH',data=df)
axs[2].scatter(x='TOTROOMS',y='KWH',data=df)
axs[0].set_xlabel('Total Cooled Square Footage')
axs[0].set_ylabel('Electricity Usage in KWH')
axs[1].set_xlabel('Total Heated Square Footage')
axs[1].set_ylabel('Electricity Usage in KWH')
axs[2].set_xlabel('Total Rooms in Housing')
axs[2].set_ylabel('Electricity Usage in KWH')

We previosuly found that total cooled square footage, total heated square footage and total rooms in housing had a moderate positive correlation with target variable 'KWH'. The scatter plots above are quite informative and are in-line with the results from correlation analysis. We can see that we do not have one fixed linear relationship across the entire domain of values of total cooled square footage, total heated square footage and total rooms in housing.

<a id="46"></a>
#### f. Detect Outliers and Anomalies

Previously, in the summary statistics section, we found that the mean value was different from the median value for the features HDD65, CDD65, HDD30YR, CDD30YR, TOTSQFT, KWH, CDD80, OA_LAT. In addition, we found a large difference in the 75th percentile and maximum value for these features. These observations indicates the presence of potential outlier values in these features. Let's check each of these feature for outliers using box plot (i.e. box and whisker diagram)

In [ ]:
outl_cols = ['HDD65', 'CDD65', 'HDD30YR', 'CDD30YR', 'TOTSQFT', 'CDD80', 'OA_LAT']
number_of_columns=len(outl_cols)
number_of_rows = len(outl_cols)-1/number_of_columns
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(outl_cols)):
    plt.subplot(round(number_of_rows + 1),number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(y = df[outl_cols[i]],orient='h')
    plt.tight_layout()

We can see from the box plot above that almost all the features shows outliers present in the dataset. Let's now check the linearity of the variables by plotting distribution graph and look for skewness of features using Kernel density estimate (kde)

In [ ]:
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(outl_cols)):
    plt.subplot(round(number_of_rows + 1),number_of_columns,i+1)
    sns.distplot(df[outl_cols[i]],kde=True)
    plt.tight_layout()

All the predictor variables depicted in kde graph above are right skewed/positively skewed

In the [Check for Missing Values](#d-check-for-missing-values) section, we found 72 predictor features that had NA i.e. 'Not Applicable' values in more than 95% of the observations. Let's now check whther these 72 features correlate with the target variable 'KWH' or not

In [ ]:
target_var = 'KWH'
feature_target_corr = {}
for col in list(na_col_names.values):
    feature_target_corr[col + '-' + target_var] = round(scipy.stats.spearmanr(df[col], df[target_var])[0],2)
print("Predictor Feature vs Target Variable Correlations")
print(feature_target_corr)

Spearman correlation coefficient values between target variable 'KWH' and 72 predictor features reveal low to negligible correlation. We could have employed imputation technique to address these 72 features, however, we can drop them due to high missing values i.e. 'Not Applicable' values and evidence of negligible correlation suggesting that the set of these features have negligible amount of predictive power over the outcome variable. Hence, these features will be removed in data preprocessing section

<a id="47"></a>
#### g. Explore Additional Predictor Variables

Let's now explore the relationship between target variable 'KWH' and categorical features 'METROMICRO', 'UR' and 'IECC_Climate_Pub' using box plots

In [ ]:
number_of_columns=len(cat_features_remaining)
number_of_rows = len(cat_features_remaining)-1/number_of_columns
plt.figure(figsize=(10*number_of_columns, 16*number_of_rows))
for i in range(0,len(cat_features_remaining)):
    plt.subplot(round(number_of_rows + 1),number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(x = df[cat_features_remaining[i]], y ='KWH', data = df)
    plt.xlabel(cat_features_remaining[i], size = 20)
    plt.ylabel('KWH', size = 20)
    plt.xticks(size = 20)
    plt.yticks(size = 20)
    plt.tight_layout()

We can observe in the box plots above that all the categories of categorical features has energy consumption value under KWH 80,000 except for just one extreme outlier value above KWH 140,000. We will take care of the outlier values in the data preprocessing section

We will convert the categorical labels of categorical features into numbers using the `OneHotEncoder` inside `ColumnTransformer` later in the model building section. It is to be noted that incase the cardinality of a particular categorical feature is very high, then using one-hot encoding is not recommended as it might lead to a curse of dimensionality. However, in our dataset we have three categorical features with feature 'IECC_Climate_Pub' having the maximum unique feature values of 11

Let's now explore how some of the housing characterstics, usage patterns and household demographics features are related with the target variable 'KWH' i.e. Total Electricity usage in KWH. The following features under housing characterstics, usage patterns and household demographics will be explored in-depth to uncover underlying patterns in the dataset:
- **Housing Characterstics:** type of housing unit (TYPEHUQ), year housing unit was built (YEARMADERANGE), total number of rooms in housing (TOTROOMS)
- **Usage Patterns:** frequency of clothes dryer use (DRYRUSE), frequency of oven use (OVENUSE), frequency of dishwasher use (DWASHUSE)
- **Household Demographics :** number of household members (NHSLDMEM), gross household income (MONEYPY)

It may be noted here that all the features falling under housing characterstics, usage patters and household demographics are discrete numerical variables i.e. the variables whose values exist in a particular range or are countable in a finite amount of time

**How does Housing Characterstics features relates with the target variable 'KWH'?**

a) Relation Between Type of Housing Unit and Energy Consumption in KWH

In [ ]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Plot
plt.figure(figsize=(20,6))

# 1. Countplot
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='TYPEHUQ', data=df)
ax0.set_title('Distribution of Housing Unit')

# 2. Boxplot
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='TYPEHUQ', y='KWH', data=df)
ax1.set_title('Relationship b/w Housing Unit Type & Energy Consumption')

# 3. Barplot (Median)
plt.subplot(1,3,3)
df.groupby('TYPEHUQ')['KWH'].median().plot.bar()
plt.title('Median KWH for each Housing Unit Type')
plt.ylabel('KWH')

plt.tight_layout()
plt.show()


Based on the above plots, we can observe the following:
- Housing unit type 2 i.e. Single-Family Detached dominates the data distribution of Housing Unit type (n = ~7800 observations)
- We may observe a clear pattern in the boxplot above that as housing unit type change from Mobile home to single-family home to apartment in building, lesser the energy is consumed
- From the plot 'Median KWH for each Housing Unit type', we can see that feature TYPEHUQ has a direct relation with the target variable

b) Relation Between Year in which Housing Unit was built and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='YEARMADERANGE', data = df)
ax0.set_title('Distribution of Housing Unit Built Year')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='YEARMADERANGE', y ='KWH', data = df)
ax1.set_title('Relationship b/w Housing Unit Built Year & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('YEARMADERANGE')['KWH'].median().plot.bar()
plt.title('Median KWH for each Housing Unit Built Year Range')
plt.ylabel('KWH')

Based on the above plots, we can observe the following:
- We can see from the barplot that Housing Unit Built Year has data well-distributed across different levels/range of built year
- We may see a clear pattern in the boxplot above that most recently the housing unit was built, more energy consumption can be observed
- 'Median KWH for each housing unit built year range' graph observation is in-line with boxplot results indicating that feature YEARMADERANGE indeed has a direct relation with the target variable

c) Relation Between Total Rooms in  Housing and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(27,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='TOTROOMS', data = df)
ax0.set_title('Distribution of Total Rooms in Housing')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='TOTROOMS', y ='KWH', data = df)
ax1.set_title('Relationship b/w Housing Unit Total Rooms & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('TOTROOMS')['KWH'].median().plot.bar()
plt.title('Median KWH for # Rooms in Housing')
plt.ylabel('KWH')

Based on the above plots, we can observe the following:
- Most of the Housing units typically have less than 12 rooms in total. Only few of the housings have more than or equal to 12 rooms in total. Hence, we could combine these levels together and create a new one 'more than 11 rooms'. We will take care of this in data preprocessing section
- We may see a clear pattern in the boxplot above that more the number of rooms in the housing, more the energy consumption is
- Similarly, the median KWH plot also show that as number of rooms in housing increases, the energy consumption increases as well, indicating that feature TOTROOMS has a direct relation with the target variable

**How does Appliance Usage Pattern features relates with the target variable 'KWH'?**

a) Relation Between Frequency of Clothes Dryer Use and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='DRYRUSE', data = df)
ax0.set_title('Distribution of Frequency of Clothes Dryer Use')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='DRYRUSE', y ='KWH', data = df)
ax1.set_title('Relationship b/w Frequency of Clothes Dryer Use & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('DRYRUSE')['KWH'].median().plot.bar()
plt.title('Median KWH for Frequency of Clothes Dryer Use')
plt.ylabel('KWH')

Based on the above plots, we can observe the following:
- Most of the households use clothes dryer everytime they wash clothes (depicted by factor/level value 1. Note that factor/level value 3 indicates infrequent use of clothes dryer)
- We can observe a clear pattern in the boxplot above that more frequently a household use clothes dryer, more is the energy consumption
- The median KWH plot results are in-line with boxplot results, indicating that more frequently household use clothes dryer, more is the median value of energy consumption. Thus, feature DRYRUSE has a direct relation with the target variable

b) Relation Between Frequency of Oven Use and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='OVENUSE', data = df)
ax0.set_title('Distribution of Frequency of Oven Use')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='OVENUSE', y ='KWH', data = df)
ax1.set_title('Relationship b/w Frequency of Oven Use & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('OVENUSE')['KWH'].median().plot.bar()
plt.title('Median KWH for Frequency of Oven Use')
plt.ylabel('KWH')

Based on the above plots, we can observe the following:
- Majority of the households use the oven 'few times a week' (depicted by factor/level value 1. Note that factor/level value 1 indicates use of oven three or four times a day)
- Contrary to the results of barplot, from the boxplot and median KWH plot, we can observe that most energy consumption was from the households who were using oven 'two times a day' (depicted by higher median value for factor/level value 2 of feature OVENUSE)

c) Relation Between Frequency of Dishwasher Use and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='DWASHUSE', data = df)
ax0.set_title('Distribution of Frequency of Dishwasher Use')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='DWASHUSE', y ='KWH', data = df)
ax1.set_title('Relationship b/w Frequency of Dishwasher Use & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('DWASHUSE')['KWH'].median().plot.bar()
plt.title('Median KWH for Frequency of Dishwasher Use')
plt.ylabel('KWH')

Based on the above plots, we can observe the following:
- From the barplot, we may observe that most of the households use their dishwashers '2 or 3 times a week' (depicted by factor/level value 13. Note that factor/level value 11 indicates use of dishwasher less than once a week and factor/level 30 indicates using dishwasher at least once each day)
- We can observe a clear pattern in the boxplot as well as median KWH plot above that more frequently household use dishwasher, more is the energy consumption. Thus, feature DWASHUSE has a direct relation with the target variable

**How does Household Demographic features relates with the target variable 'KWH'?**

a) Relation Between Total Members in Household and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(24,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='NHSLDMEM', data = df)
ax0.set_title('Distribution of Total Members in Household')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='NHSLDMEM', y ='KWH', data = df)
ax1.set_title('Relationship b/w Total Members in Household & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('NHSLDMEM')['KWH'].median().plot.bar()
plt.title('Median KWH for Total Members in Household')
plt.ylabel('KWH')

Based on the results of above bar plot, we can observe that very less number of households had more than 6 members in the household. The barplot and median KWH plot doesn't indicate a clear pattern of relationship between total members in household and energy consumption. However, if combine factors/levels 6-14 of feature NHSLDMEM together and create a new one 'more than 5 members, then we might observe a pattern indicating that as members in the household increases, the energy consumption increases as well. We will combine the levels in data preprocessing section

b) Relation Between Gross Household Income and Energy Consumption in KWH

In [ ]:
plt.figure(figsize=(27,6))
plt.subplot(1,3,1)
sns.set_style('whitegrid')
ax0 = sns.countplot(x='MONEYPY', data = df)
ax0.set_title('Distribution of Gross Household Income')
plt.subplot(1,3,2)
ax1 = sns.boxplot(x='MONEYPY', y ='KWH', data = df)
ax1.set_title('Relationship b/w Gross Household Income & Energy Consumption')
plt.subplot(1,3,3)
df.groupby('MONEYPY')['KWH'].median().plot.bar()
plt.title('Median KWH for each category of Gross Household Income')
plt.ylabel('KWH')

Based on the results of above bar plot, we can observe that very less number of households had less than \$10,000 as their gross household income (NOTE: factor/level 5 indicates gross household income of \$10,000 to \$14,999). The barplot and median KWH plot indicates higher energy consumption for households with gross income less than \$2,500 (depicted by factor/level 1) as compared to households with gross income between \$2,500 to \$19,999 (i.e. factor/level 2 to 6). This could be because of free electricity subsidy programs by state and federal government. However, for households with gross income greater than \$20,000, a clear pattern can be observed that as gross income of a household increase overall, the energy consumption increases as well

<a id="5"></a>
### 5. Data Transformation and Preprocessing

<a id="51"></a>
#### a. Data Transformation

<a id="511"></a>
##### i. Combining/Merging Predictor Features

Let's create a couple of new features by combining some of the existing features. These new features alongwith the existing ones will be later analyzed in feature selection section to determine whether keeping these features would help to understand our target variable better.

Here's the list of new features that will be created in this section:
- **TV Equipment Features:** We will merge all the TV equipment related features (such as TV, VCR, DVD, game console, home theatre, cable box and set-top box) and create a new feature called ALLTVFTR. The features that we will combine are ['TVCOLOR', 'CABLESAT1', 'COMBODVR1','DVR1', 'DIGITSTB1', 'PLAYSTA1', 'COMBOVCRDVD1', 'VCR1', 'DVD1', 'TVAUDIOSYS1', 'OTHERSTB1', 'CABLESAT2', 'COMBODVR2', 'DVR2', 'DIGITSTB2', 'PLAYSTA2', 'COMBOVCRDVD2', 'VCR2', 'DVD2', 'TVAUDIOSYS2', 'OTHERSTB2', 'CABLESAT3', 'COMBODVR3', 'DVR3', 'DIGITSTB3', 'PLAYSTA3', 'COMBOVCRDVD3', 'VCR3', 'DVD3', 'TVAUDIOSYS3', 'OTHERSTB3']

- **Office Equipment Features:** We will merge all the Office equipment related features (such as computer, monitor, printer, fax machine and cope machine) and create a new feature called ALLOFFFTR. The features that we will combine are ['NUMPC', 'PCPRINT', 'FAX', 'COPIER', 'MONITOR1', 'MONITOR2', 'MONITOR3']

- **Telephone Features:** We will merge all the telephone equipment related features (such as cordless telephone and answering machine) and create a new feature called ALLTELFTR. The features that we will combine are ['NOCORD', 'ANSMACH']

Let's just first start by replacing -2 value in all the above mentioned features to 0. Changing -2 to 0 in above features will ensure that we are not subtracting any value during the merging of these features. NOTE: None of the telephone features has -2 factor/level


In [ ]:
tv_features = ['TVCOLOR', 'CABLESAT1', 'COMBODVR1','DVR1', 'DIGITSTB1', 'PLAYSTA1', 'COMBOVCRDVD1', 'VCR1', 'DVD1', 'TVAUDIOSYS1', 'OTHERSTB1', 'CABLESAT2', 'COMBODVR2', 'DVR2',
'DIGITSTB2', 'PLAYSTA2', 'COMBOVCRDVD2', 'VCR2', 'DVD2', 'TVAUDIOSYS2', 'OTHERSTB2', 'CABLESAT3', 'COMBODVR3', 'DVR3', 'DIGITSTB3', 'PLAYSTA3', 'COMBOVCRDVD3', 'VCR3', 'DVD3',
'TVAUDIOSYS3', 'OTHERSTB3']
office_features = ['NUMPC', 'PCPRINT', 'FAX', 'COPIER', 'MONITOR1', 'MONITOR2', 'MONITOR3']
tel_features = ['NOCORD', 'ANSMACH']

for i in tv_features:
    df[i] = df[i].apply(lambda x : x if x != -2 else 0)
df['ALLTVFTR'] = df[tv_features].sum(axis=1)

for i in office_features:
    df[i] = df[i].apply(lambda x : x if x != -2 else 0)
df['ALLOFFFTR'] = df[office_features].sum(axis=1)

df['ALLTELFTR'] = df[tel_features].sum(axis=1)

<a id="512"></a>
##### ii. Combining/Merging Levels with Low Frequency in Discrete Predictor Features

In the [Explore Additional Predictor Variables](#g-explore-additional-predictor-variables) section of EDA, we observed that some of the discrete numeric variables 'Total Rooms in Housing (TOTROOMS) and 'Total Members in Household' (NHSLDMEM) had only few data observations beyond a specific factor value. Hence, we will merge some of the levels for these variables due to the low frequency of values for these levels. In particular, for feature TOTROOMS, we will merge values 15 to 23 and create a new level 12 which would imply 'More than 14 Rooms'. For feature NHSLDMEM, we will merge values 6 to 14 to create a new level 6 which would mean 'More than 5 Members'

In [ ]:
df['TOTROOMS'] = df['TOTROOMS'].replace([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [12]*12)
df['NHSLDMEM'] = df['NHSLDMEM'].replace([6, 7, 8, 9, 10, 11, 12, 13, 14], [6]*9)

Let's verify whether the levels have been merged or not

In [ ]:
print(df['TOTROOMS'].describe(), df['TOTROOMS'].value_counts())

In [ ]:
print(df['NHSLDMEM'].describe(), df['NHSLDMEM'].value_counts())

<a id="52"></a>
#### b. Data Preprocessing

<a id="521"></a>
##### i. Removing Predictor Features with high 'Not Applicable' Values

Previosuly, in the [Check for Missing Values](#d-check-for-missing-values) section of EDA we found 72 features with 'Not Applicable' values in more than 95% of the observations (i.e. less variability in the data values since >95% of observations are same value -2 i.e. NA). And these features were also checked for their correlation with our target variable and thus on the basis of high NA values and very weak correlation with target variable, we concluded that these 72 features have negligible amount of predictive power over the outcome variable. Hence, we will now drop these features from our dataset.

In addition, in the [Summary Statistics](#c-summary-statistics) section of EDA, we also found two other features 'NOCRCASH' and 'NKRGALNC' to have more than 99% of the data values marked as -2 i.e. 'Not Applicable'. Hence, we will drop these features too since there is less variability in the data values and thus these features might not have any influence over the outcome variable

In [ ]:
df.shape # let's print the shape before dropping above discussed 74 features

In [ ]:
highNA_features = list(na_col_names.values)
highNA_features.extend(['NOCRCASH', 'NKRGALNC']) # adding two more features to the list of features with high NA values
print(len(highNA_features))

In [ ]:
df.drop(highNA_features, axis = 1, inplace = True)

In [ ]:
df.shape # let's print the shape after dropping above discussed 74 features.

<a id="522"></a>
##### ii. Removing Imputation Flags

All the features in our dataset starting with 'Z' are features representing imputation flags and we will now remove all of these features below.

In [ ]:
# Let's make a list of all the features starting with 'Z'
Z_features = [col for col in df if col.startswith('Z')]
print('We have {} imputation flag features'.format(len(Z_features)))
print('-----'*38)
print('These features are: \n{}'.format(Z_features))

In [ ]:
df.shape # let's print the shape before dropping above mentioned 359 features.

In [ ]:
df.drop(Z_features, axis = 1, inplace = True)

In [ ]:
df.shape # let's print the shape after dropping the imputation flag features.

<a id="523"></a>
##### iii. Removing Duplicate Features

Previously, in the [Explore Target Variable and Extract Important Features](#e-explore-target-variable-and-extract-important-features) section of EDA, we found that some features present in the dataset to be duplicates such as KWH and BTUEL are both energy consumption features but in different units. Hence, we will drop all BTU features. In addition, we will drop price/cost features too since it is only consumption we are interested in. Moreover, cost is usually determined from the energy usage. Let's now make a list of all the BTU and DOL features that needs to be removed. Below is a list of all BTU, KWH and cost (columns with keyword DOL) related features. Some of these features were compiled by summing up one or more predictor features (as can be seen below:
)
- KWH = KWHSPH + KWHCOL + KWHWTH + KWHRFG + KWHOTH<br>
- BTUEL = BTUELSPH + BTUELCOL + BTUELWTH + BTUELRFG + BTUELOTH<br>
- DOLLAREL = DOLELSPH + DOLELCOL + DOLELWTH + DOLELRFG + DOLELOTH<br>
- CUFEETNG = CUFEETNGSPH + CUFEETNGWTH + CUFEETNGOTH<br>
- BTUNG = BTUNGSPH + BTUNGWTH + BTUNGOTH<br>
- DOLLARNG = DOLNGSPH + DOLNGWTH + DOLNGOTH<br>
- GALLONLP = GALLONLPSPH + GALLONLPWTH + GALLONLPOTH<br>
- BTULP = BTULPSPH + BTULPWTH + BTULPOTH<br>
- DOLLARLP = DOLLPSPH	+ DOLLPWTH + DOLLPOTH<br>
- GALLONFO = GALLONFOSPH + GALLONFOWTH + GALLONFOOTH<br>
- BTUFO = BTUFOSPH + BTUFOWTH + BTUFOOTH<br>
- DOLLARFO = DOLFOSPH + DOLFOWTH + DOLFOOTH<br>
- GALLONKER = GALLONKERSPH + GALLONKERWTH + GALLONKEROTH<br>
- BTUKER = BTUKERSPH + BTUKERWTH + BTUKEROTH<br>
- DOLLARKER = DOLKERSPH + DOLKERWTH + DOLKEROTH<br>
- TOTALBTU = TOTALBTUSPH + TOTALBTUCOL + TOTALBTUWTH + TOTALBTURFG + TOTALBTUOTH<br>
- TOTALDOL = TOTALDOLSPH + TOTALDOLCOL + TOTALDOLWTH + TOTALDOLRFG + TOTALDOLOTH<br>
- TOTALBTUSPH = BTUELSPH + BTUNGSPH + BTULPSPH + BTUFOSPH + BTUKERSPH<br>
- TOTALBTUCOL = BTUELCOL<br>
- TOTALBTUWTH = BTUELWTH + BTUNGWTH + BTULPWTH +  + BTUFOWTH + BTUKERWTH<br>
- TOTALBTURFG = BTUELRFG<br>
- TOTALBTUOTH = BTUELOTH + BTUNGOTH + BTULPOTH + BTUFOOTH + BTUKEROTH<br>
- TOTALDOLSPH = DOLELSPH + DOLNGSPH + DOLLPSPH + DOLFOSPH + DOLKERSPH<br>
- TOTALDOLCOL = DOLELCOL<br>
- TOTALDOLWTH = DOLELWTH + DOLNGWTH + DOLLPWTH + DOLFOWTH + DOLKERWTH<br>
- TOTALDOLRFG = DOLELRFG<br>
- TOTALDOLOTH = DOLELOTH + DOLNGOTH + DOLLPOTH + DOLFOOTH + DOLKEROTH<br>

In [ ]:
BTU_features = [col for col in df if col.startswith('BTU')]
# Let's also add TOTALBTU features to this list
BTU_features.extend([col for col in df if col.startswith('TOTALBTU')])
print('We have {} BTU features'.format(len(BTU_features)))
print('-----'*38)
print('These features are: \n{}'.format(BTU_features))

In [ ]:
df.shape # let's print the shape before dropping above mentioned 29 BTU features.

In [ ]:
df.drop(BTU_features, axis = 1, inplace = True)

In [ ]:
df.shape # let's print the shape after dropping the BTU features. Expected Shape:

Let's now make a list of all cost/price features i.e. features starting with keywords 'DOL' and 'TOTALDOL'

In [ ]:
DOL_features = [col for col in df if col.startswith('DOL')]
# Let's also add TOTALDOL features to this list
DOL_features.extend([col for col in df if col.startswith('TOTALDOL')])
print('We have {} DOL or cost features'.format(len(DOL_features)))
print('-----'*38)
print('These features are: \n{}'.format(DOL_features))

In [ ]:
df.shape # let's print the shape before dropping above mentioned 28 DOL features.

In [ ]:
df.drop(DOL_features, axis = 1, inplace = True)

In [ ]:
df.shape # let's print the shape after dropping the DOL features.

Let's now drop the features related to Wood usage, LPG/Propane usage, Natural Gas usage, Fuel Oil usage and Kerosene usage. In addition, we will also drop the following KWH features 'KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH' since our target variable is KWH

In [ ]:
othr_usage_features = ['KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'CUFEETNG', 'CUFEETNGSPH', 'CUFEETNGWTH', 'CUFEETNGOTH', 'GALLONLP', 'GALLONLPSPH',
'GALLONLPWTH', 'GALLONLPOTH', 'GALLONFO', 'GALLONFOSPH', 'GALLONFOWTH', 'GALLONFOOTH', 'GALLONKER', 'GALLONKERSPH', 'GALLONKERWTH', 'GALLONKEROTH', 'CORDSWD']

print('We have {} other usage and additional KWH features'.format(len(othr_usage_features)))
print('-----'*38)
print('These features are: \n{}'.format(othr_usage_features))

In [ ]:
df.shape # let's print the shape before dropping above mentioned 22 other usage and additional KWH features.

In [ ]:
df.drop(othr_usage_features, axis = 1, inplace = True)

In [ ]:
df.shape # let's print the shape after dropping the other usage and additional KWH features.

Let's drop the feature DOEID as it is identifer rather than predictor. In addition, let's also drop NWEIGHT

In [ ]:
df.drop(['DOEID', 'NWEIGHT'], axis = 1, inplace = True)
# Let's print the shape after droping these features.
df.shape

<a id="524"></a>
##### iv. Removing Outliers

Previously, in the [Explore Target Variable and Extract Important Features](#e-explore-target-variable-and-extract-important-features) section of EDA, we found that almost all the observations had KWH value under 80,000. Let's check how many observations in the dataset have KWH greater than 80,000 and accordingly handle these outlier values (if any)

In [ ]:
# Let's check how many records have KWH > 80,000
print(f"Number of records that have more than 80000 KWH: {df[df.KWH > 80000].shape}")

As we can see above, only one record has been found to have more than 80000 KWH value and in the previous section we saw that this oulier value of KWH was indeed very high to be true. Hence, we will now drop this row from our dataset

In [ ]:
df2 = df[df.KWH <= 80000]
df2.shape

Let's also verify visually from boxplot whether this extreme outlier value has been excluded or not

In [ ]:
sns.set(style="darkgrid")
fig, ax = plt.subplots(figsize = (10,6))
ax = sns.boxplot(x=df2['KWH']).set(title = 'KWH Boxplot')

<a id="6"></a>
### 6. Feature Engineering

<a id="61"></a>
#### a. Exploratory Feature Reduction

Let's use Principle Component Analysis (PCA), a feature reduction technique to see how many features can be used for data modeling. In other words, PCA is a technique to obtain important features from a large set of features which explains the most of the variability in the data. Let's start implementing PCA by first removing response variable from the dataset.

In [ ]:
df_tmp = df2.drop(['KWH', 'KWHlog'], axis = 1)
df_tmp.shape

Let's use One-hot Encoding, a feature encoding strategy, first to convert our categorical features into a numerical feature

In [ ]:
df_tmp = pd.get_dummies(df_tmp, columns= cat_features_remaining, drop_first = True)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# Convert to numpy and scale
X = scale(df_tmp.values)

# Use a valid number of components (≤ min(n_samples, n_features))
pca = PCA(n_components=428)
pca.fit(X)


In [ ]:
var= pca.explained_variance_ratio_

Let's build a scree plot i.e. a line plot that shows the eigenvalues for each individual principal component. Scree plot helps us to access components or factors which explains the most of variability in the data. It represents values in descending order.

In [ ]:
plt.grid(which='major', linestyle='--', linewidth='0.5', color='grey')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.plot(var)

The above scree plot indicates that approx. 20 principal components were able to capture most of the information.

In [ ]:
#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

The above cumulative variance can be read as follows:
It shows that first principal component explains 10.1% variance. The first and second component cumulatively explains 15.05% variance or we can say that second principal component alone explains 4.95% variance. First, second and third component cumulatively explains 19.05% variance and so on. Let's now plot cumulative variance

In [ ]:
# cumulative scree plot
plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
plt.title('Cumulative Variance Plot')
plt.xlabel('Number of Features')
plt.ylabel('Cumulative Proportion of Variance Explained')
plt.plot(var1)

The above plot shows that ~ 200 features result in variance close to ~ 95%. The PCA analysis gives us a ballpark estimate of the number of features that explains majority of the variation in the dataset and hence can be used for data modeling. However, let's use Gradient Boosting Machine (GBM) in the later [Feature Selection](#) section to automatically calculate feature importance

<a id="62"></a>
#### b. Feature Selection

I am going to use a pre built `FeatureSelector` class made available by Will Koehrsen on their [GitHub](https://github.com/WillKoehrsen/feature-selector/blob/master/feature_selector/feature_selector.py). This `FeatureSelector` includes the following most common feature selection methods:

- Features with a high percentage of missing values
- Collinear (highly correlated) features
- Features with zero importance in a tree-based model
- Features with low importance
- Features with a single unique value

Earlier, during the EDA phase of the project, we found that there were indeed no missing values in the RECS 2009 dataset. Hence, the `FeatureSelector` class that we are going to utilize for Feature Selection phase of the project, we are only going to use the last four common feature selection methods. Let's now start by finding features with single unique value

<a id="621"></a>
##### i. Find Features with Single Unique Value


In [ ]:
labels = df2['KWH']
data_fs = df2.drop(columns = ['KWH', 'KWHlog'])

In [ ]:
fs = FeatureSelector(data = data_fs, labels = labels) # Let's inititate the class FeatureSelector()

In [ ]:
fs.identify_single_unique()
single_unique = fs.ops['single_unique']
single_unique

As we can see above, we just have one feature 'USEEL' in the entire set of 441 features with single unique value. Let's now check the collinear features in our dataset. Collinear features can be defined as the features that are highly correlated with one another. In practice, we exclude the features which are highly correlated as including a pair of such features can lead to decrease in model performance on test set.

<a id="622"></a>
##### ii. Find Collinear Features

In [ ]:
fs.identify_collinear(correlation_threshold=0.98)
correlated_features = fs.ops['collinear']
print('----------------------------------------------------------------------------------------')
print('These features are: \n{}'.format(correlated_features))

Let's now plot all the correlations which were found to be above threshold using correlation heatmap of the correlation values

In [ ]:
features = list(set(fs.record_collinear['corr_feature']).union(set(fs.record_collinear['drop_feature'])))
valid_features = [f for f in features if f in df2.columns]
corr_mat_plt = df2[valid_features].corr()

Let's also check which pair of features were found to be correlated with each other and their respective correlation values

In [ ]:
fs.record_collinear

<a id="623"></a>
##### iii. Find Features with Zero Importance using GBM

In [ ]:
fs.identify_zero_importance(task = 'regression', eval_metric = 'l2', n_iterations = 10, early_stopping = False)
print('----------------------------------------------------------------------------------------')
print('These features are: \n{}'.format(fs.ops['zero_importance']))

The above method of feature selection has been designed for machine learning problem. Using Gradient Boosting Machine learning model from the [LightGBM Library](https://lightgbm.readthedocs.io/en/v3.3.2/), we tried to find the features from our dataset which have zero importance. These features were averaged over 10 training runs in order to reduce variance. As we can observe from the above results, 28 features were found to have zero importance and we can remove these features later on without affecting model performance.

Let's now check how many features have a cumulative importance of 90% and also see the top 20 features in order of their importance.

In [ ]:
fs.plot_feature_importances(threshold = 0.9, plot_n = 20)

As can be seen from the first plot, among the top 20 features (plotted in terms of normalized importance), the top five of them are YEARMADE, HHAGE, TOTCSQFT, CDD80 and CDD30YR. The plot at the bottom shows cumulative importance ploteed on y-axis and number of features on x-axis. The vertical dotted line is drawn at the threshold value of cumulative importance we chose above i.e. 90%. We can notice that 225 features are required for 90% of the cumulative importance.

In [ ]:
top_features = list(fs.feature_importances.loc[:224, 'feature'])
print(top_features)

<a id="624"></a>
##### iv. Find Features with Low Importance

Let's now find the lowest importance features that do not contribute to a specified total importance. Recall that when finding important features in the section above, we used a cumulative importance threshold of 90%. Let's use the same threshold value to find the least important features that are not required to achieve 90% of total importance

In [ ]:
fs.identify_low_importance(cumulative_importance = 0.90)

As you can notice, based on the cumulative importance threshold value of 90%, the gradient boosting machine considers 217 features to be not relevant for model learning purpose.
<a id="625"></a>
##### v. Removing Features

Using above four methods in [Feature Selection](#b-feature-selection), we identifed which features to drop. Now' let's drop all these features

In [ ]:
data_fs_removed = fs.remove(methods = 'all')

In [ ]:
print(list(data_fs_removed.columns))
print(len(data_fs_removed.columns))

Let's remove the one hot encoding features from the above `data_fs_removed` dataframe i.e. we will remove all the columns which were categorical in nature and were converted to one hot encodings during the process of feature selection. These columns are METROMICRO_METRO, UR_R and IECC_Climate_Pub_3A. The reason we are doing this is because we ran feature selection on the entire training set without separating the test set. In reality, while training the model, we would separate the dataset into two subsets, training and test. The training set will be further divided into two splits during cross validation, training and validation fold, and we don't have any information beforehand whether we would have categories in the test data that were not in the training data. Usually, if such case arise, an error will occur. In addition, another way to think about this is considering the deployment stage of the model. There is a chance that the data distributions in future might change and we might get new categories in our categorical data features and the model prediction will result in error if there are new categories in the dataset. Hence, we would use the `ColumnTransformer` inside the `Pipelines` in the later section of model development to handle the categorical features.

However, it's worth mentioning here that of all the categories of the categorical features, only three, METROMICRO_METRO, UR_R and IECC_Climate_Pub_3A, were found to be important features by the `FeatureSelector` class

In [ ]:
data_fs_removed.shape

Now, let's add original categorical feature names to the list of columns of data_fs_removed

In [ ]:
print('Original Number of Features', data_fs.shape[1])
print('Final Number of Features: ', data_fs_removed.shape[1])

Let's append our target variable column to final_columns list

In [ ]:
final_columns = list(data_fs_removed.columns)
final_columns.append("KWH")

<a id="7"></a>
### 7. Model Development & Comparison
<a id="71"></a>
#### a. Building Baseline Models with default params

In [ ]:
final_data = df2[final_columns]
final_data.shape

In [ ]:
# Separate features and labels
y = final_data['KWH']
X = final_data.drop(columns =['KWH'])

We could train a model using all the data we have; however it is a common practice in supervised machine learning to split into two subsets; a larger set with which to train the model, and a smaller holdout data set (also called test set) to provide an unbiased evaluation of a final model fit on the training data set.

Next we do the train-test split and hold out the test set until we decide our final model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
print(f'X_train shape is:{X_train.shape} \nX_test shape is:{X_test.shape}\ny_train shape is:{y_train.shape}\ny_test shape is:{y_test.shape}')

Next, we normalize the numeric features using `StandardScaler()` to transform our feature data such that its distribution will have a mean value 0 and standard deviation of 1. Normalizing is an important step in machine learning as it brings all the features on the same scale and thus prevents features with large values from producing coefficients that disproportionately affect the predictions. In addition to scaling transformation, we also need to apply one hot encoding to our categorical features to convert categories into numbers since scikit-learn only accepts numeric data as input. We will make use of `ColumnTransformer` by defining the separate preprocessing pipelines, each for numeric and categorical features.

We will then wrap the column transformer in another pipeline containing our regressor using the `Pipeline` utility available in sklearn and finally use this pipeline inside `cross_validate`

In [ ]:
numeric_features = X_train.select_dtypes('number').columns
categorical_features = X_train.select_dtypes('object').columns

In [ ]:
numeric_transformer = Pipeline(
    steps = [("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps = [("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

col_transformer = ColumnTransformer(
    transformers = [
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features)
]
)

#### Train Baseline Models with Default Params

Now we start training baseline model using default hyperparameters. We will use cross validation process in model training. If we were to fit the model with the training set while evaluated with the test set, we obtained only a single sample point of evaluation with one test set. How can we be sure it is an accurate evaluation, rather than a value too low or too high by chance? If we have two models, and found that one model is better than another based on the evaluation, how can we know this is also not by chance?

Hence, we make use of cross validation to evaluate each model multiple times with different dataset and take the average score for our decision to choose the final model candidate for evaluation on holdout dataset or test dataset. __Cross validation__ uses k-fold to resample the same dataset multiple times and pretend they are different. With cross validation, as we are evaluating the model, or hyperparameter, the model has to be trained from scratch, each time, without reusing the training result from previous attempts.

In [ ]:
pipeline = []
pipeline.append(("Linear Regression", Pipeline([("preprocessor", col_transformer), ("LR", LinearRegression())])))
pipeline.append(("Lasso", Pipeline([("preprocessor", col_transformer), ('Lasso', Lasso())])))
pipeline.append(("Ridge", Pipeline([("preprocessor", col_transformer), ('Ridge', Ridge())])))
pipeline.append(("ElasticNet", Pipeline([("preprocessor", col_transformer), ('eNet', ElasticNet())])))
pipeline.append(("RForest", Pipeline([("preprocessor", col_transformer), ('RF', RandomForestRegressor())])))
pipeline.append(("Gradient Boosting", Pipeline([("preprocessor", col_transformer), ('GBM', GradientBoostingRegressor())])))
pipeline.append(("XG Boost", Pipeline([("preprocessor", col_transformer), ('xgb', xgb.XGBRegressor(objective = "reg:squarederror"))])))

Let's define the scoring criteria by selecting:
- __Root Mean Square Error (RMSE):__ The square root of the mean of the squared difference between predicted and actual values. This yields an absolute metric in the same unit as the label (in this case, Electricity Consumption in KWH). The smaller the value, the better the model is (i.e. in a simplistic sense, it represents the average electricity consumption by which the predictions are wrong!)

- __Coefficient of Determination (usually known as R-squared or R2):__ Higher the value of this metric, the better the fit of the model is. This metric represents how much of the variance between predicted and actual label values the model is able to explain. The R-squared metric might not be considered to be a good metric in our case because R-square value increases artificially as the number of features increases. Hence, we will set RMSE to be the main scoring criteria later in the hyperparameter tuning section

You may use other metrics for evaluation regression models. Refer this [Scikit-Learn documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

In [ ]:
RMSE = []
R2 = []
names = []
scoring = {'rmse': 'neg_root_mean_squared_error',
           'r2': 'r2'
           }

In [ ]:
for name, model in pipeline:
    kfold = KFold(n_splits = 5, random_state = 1, shuffle = True)
    cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring, n_jobs = 1)
    RMSE.append(cv_results['test_rmse']*-1)
    R2.append(cv_results['test_r2'])
    names.append(name)

In [ ]:
avg_RMSE = [sum(i)/len(i) for i in RMSE]
avg_R2 = [sum(j)/len(j) for j in R2]
model_baseline_metrics = pd.DataFrame({
                                            'Model': names,
                                            'Baseline_AvgRMSE': avg_RMSE,
                                            'Baseline_AvgR2': avg_R2
                                        })
model_baseline_metrics

In [ ]:
plt.figure(figsize=(21,9))

ax1 = plt.subplot(1,2,1)
sns.boxplot(data = RMSE)
ax1.set_xticklabels(names)
ax1.set_ylabel("Root Mean Square Error (RMSE)")
ax1.set_title("Baseline Model RMSE comparsion across 7 models")

ax2 = plt.subplot(1,2,2)
sns.boxplot(data = R2)
ax2.set_xticklabels(names)
ax2.set_ylabel("r2 score")
ax2.set_title("Baseline Model r2 comparsion across 7 models")

plt.show()

Above, we evaluated each of the models multiple times with different dataset using cross validation process. From the dataframe and box plots above, we can see that the baseline models more or less perform similar. Let's re-run cross validation after tuning the model hyperparameters

<a id="72"></a>
#### b. Hyperparameter Tuning & Model Comparison

In [ ]:
pipeline_tuned_models = []
pipeline_tuned_models.append(("Lasso", Pipeline([("preprocessor", col_transformer), ('Lasso', Lasso(random_state = 123))])))
pipeline_tuned_models.append(("Ridge", Pipeline([("preprocessor", col_transformer), ('Ridge', Ridge(random_state = 123))])))
pipeline_tuned_models.append(("ElasticNet", Pipeline([("preprocessor", col_transformer), ('eNet', ElasticNet(random_state = 123))])))
pipeline_tuned_models.append(("RForest", Pipeline([("preprocessor", col_transformer), ('RF', RandomForestRegressor(random_state = 123))])))
pipeline_tuned_models.append(("Gradient Boosting", Pipeline([("preprocessor", col_transformer), ('GBM', GradientBoostingRegressor(random_state = 123))])))
pipeline_tuned_models.append(("XG Boost", Pipeline([("preprocessor", col_transformer), ('xgb', xgb.XGBRegressor(objective = "reg:squarederror", random_state = 123))])))

Next, we define hyperparameters we want to tune for each of the models. Note: We only tuned few hyperparameters; however if you are not constrained by computational resources you may try more hyperparameters or more settings of hyperparameters to tune

In [ ]:
# Initiaze the hyperparameters for each dictionary
param1 = {}
param1['Lasso__alpha'] = np.linspace(0, 0.2, 21)

param2 = {}
param2['Ridge__alpha'] = np.linspace(0, 0.2, 21)

param3 = {}
param3['eNet__max_iter'] = [1, 5, 10]
param3['eNet__alpha'] = np.linspace(0, 0.2, 21)
param3['eNet__l1_ratio'] = np.arange(0.0, 1.0, 0.1)

param4 = {}
param4['RF__n_estimators'] = [50, 100, 150]
max_depth = [5, 10, 15]       ##[int(x) for x in np.linspace(5, 15, num = 3)]
max_depth.append(None)
param4['RF__max_depth'] = max_depth
param4['RF__min_samples_split'] = [2, 5, 10]
# param4['min_samples_leaf'] = [5, 10, 15]  ## if you have enough computing respurces, you could try to tune more hyperparameters by uncommenting these lines
# param4['bootstrap'] = [True, False]
param4['RF__max_features'] = [1, 'sqrt']
# param4['max_leaf_nodes'] = [5, 10, 15]

param5 = {}
param5['GBM__learning_rate'] = [0.01, 0.05, 0.1, 1.0]
param5['GBM__n_estimators'] = [50, 100, 150]

param6 = {}
param6['xgb__learning_rate'] = [0.01, 0.05, 0.1]
param6['xgb__n_estimators'] = [100, 300, 600, 900]
param6['xgb__max_depth'] = [8]
param6['xgb__subsample'] = [0.7]
param6['xgb__colsample_bytree'] = [0.9]
param6['xgb__verbosity'] = [0]

In [ ]:
params = [param1, param2, param3, param4, param5, param6]
names_tunedmodels = []
best_params = []
best_score = []
best_estimator = []
avg_RMSE = []
RMSE_tunedmodels = []
avg_R2 = []
R2_tunedmodels = []
for param, (name_t, pipe) in zip(params, pipeline_tuned_models):
    gridsearch = GridSearchCV(pipe, param_grid=param, scoring=scoring, cv=5, n_jobs=-1, refit='rmse', verbose=1)
    gridsearch.fit(X_train, y_train)
    best_params.append(gridsearch.best_params_)
    best_score.append(gridsearch.best_score_)
    best_estimator.append(gridsearch.best_estimator_)
    avg_RMSE.append(np.mean(gridsearch.cv_results_['mean_test_rmse']*-1))
    avg_R2.append(np.mean(gridsearch.cv_results_['mean_test_r2']))
    RMSE_tunedmodels.append(gridsearch.cv_results_['mean_test_rmse']*-1)
    R2_tunedmodels.append(gridsearch.cv_results_['mean_test_r2'])
    names_tunedmodels.append(name_t)
list_of_models = list(zip(names_tunedmodels, best_params, best_estimator, best_score, avg_RMSE, avg_R2))
df_models_tuned = pd.DataFrame(list_of_models, columns = ['Model', 'best_params', 'best_estimator', 'best_score_rmse', 'AvgRMSE', 'AvgR2'])
df_models_tuned

In [ ]:
plt.figure(figsize=(21,9))

ax1 = plt.subplot(1,2,1)
sns.boxplot(data = RMSE_tunedmodels)
ax1.set_xticklabels(names_tunedmodels)
ax1.set_ylabel("Root Mean Square Error (RMSE)")
ax1.set_title("Tuned Model RMSE comparsion across 6 models")

ax2 = plt.subplot(1,2,2)
sns.boxplot(data = R2_tunedmodels)
ax2.set_xticklabels(names_tunedmodels)
ax2.set_ylabel("r2 score")
ax2.set_title("Tuned Model r2 comparsion across 6 models")

plt.show()

From the above rsult of cross validation using `GridSearchCV` (refer dataframe and box plots above), we can clearly see that the tree based models (i.e. Gradient Boosting Regressor and Xtreme Gradient Boosting XGB) have better RMSE compared to linear models. However; XGB model has the lowest RMSE in comparison to Gradient Boosting Regressor. Hence, we may conclude that XGB is better than GBM. For now, since XGB model is better than other models, we will retrain the model again using the set of best parameters of XGB that `GridSearchCV` found.

The reason for retraining the model is that during the cross validation we do not have a lot of data, and the smaller dataset we used previously, had a part of it held out for validation. We believe that combining the training and validation dataset can produce a better model. Hence, we retrain the model of the entire training dataset this time and evaluate the model on our holdout dataset i.e. test dataset. Because this is unseen data, it can help us evaluate the generalization, or out-of-sample, error. This should simulate what the model will do when we deploy it. We do not expect this evaluation score to be very different from that we obtained from cross validation in the previous step, if we did the model training correctly. This can serve as a confirmation for our model selection.

__NOTE:__ The dataset for evaluation on test dataset, and the one we used in cross validation, are different because we do not want data leakage. If they were the same, we would see the same score as we have already seen from the cross validation. After retraining the model, we will use test dataset. Since we used refit = 'rmse' inside `GridSearchCV`, the best estimator has already been refitted using the best found parameters on the whole training dataset. As a next step, we just need to call predict on `gridsearch.best_estimator_` using X_test dataset i.e. holdout because the 'best_estimator' is a pipeline containing both the `ColumnTransformer` and the trained model that had the best score.

<a id="73"></a>
#### c. Model Evaluation on Unseen Data

In [ ]:
# Get Predictions
predictions = gridsearch.best_estimator_.predict(X_test)

# Display Metrics
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

In [ ]:
# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Energy Consumption Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
import os
import joblib

# Ensure 'models/' folder exists
os.makedirs("models", exist_ok=True)

# Save the model
filename = 'models/xgb.pkl'
joblib.dump(gridsearch.best_estimator_, filename)
